In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from dotenv import load_dotenv

load_dotenv()

cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=500,
    return_messages=True
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to determine if any of the text is relevant to answering the question. Return any relevant text verbatim. This part of the document could be crucial for answering the question. Review documentation with a human focus
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs ["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            { "context": doc.page_content, "question": question}
    ).content
    for doc in documents
    )



map_chain = {"documents":retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Based on the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, simply say that you don't know. Do not attempt to fabricate an answer. When responding, consider that specific parts of the document might be crucial for providing an accurate answer.
            ------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain =  {"context": map_chain,"question": RunnablePassthrough(), "history": load_memory} | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

chain.invoke("Is Aaronson guilty?")

AIMessage(content='Based on the information provided, it is not possible to determine if Aaronson is guilty. The text does not mention Aaronson in relation to the crimes or the photograph that disproved their guilt.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 192, 'total_tokens': 231}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [3]:
chain.invoke("What message did he write in the table?")

AIMessage(content='The message he wrote on the table was: "DOWN WITH BIG BROTHER."', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 206, 'total_tokens': 223}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [4]:
chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character in the text who has a significant relationship with Winston, the protagonist. She is portrayed as having changed physically, with a sallow face, a scar across her forehead and temple, and a thicker, stiffened waist. Julia is depicted as having moments of contempt and admitting to betraying Winston, showcasing a complex dynamic in their relationship.', response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 274, 'total_tokens': 346}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})